In [1]:
import cobra
import pandas as pd
import matplotlib.pyplot as plt

# Read growth rate data

In [2]:
all_mu = pd.read_csv('data/extracted_growth_rates_with_media.csv', index_col=0)
# select area method (we could have used others, but area works well)"
method = "area"
all_mu = all_mu[all_mu['method'].str.contains("area")]
pca_gluc_mu = all_mu[all_mu['medium'] == "PCA-Gluc"].reset_index()
gluc_mu = all_mu[all_mu['medium'] == "Glucose-MOPS"].reset_index()
citr_mu = all_mu[all_mu['medium'] == "Citrat-MOPS"].reset_index()

mean_pca_gluc_mu = float(pca_gluc_mu['mu'].mean())
std_pca_gluc_mu = float(pca_gluc_mu['mu'].std())
print('PCA-Gluc mu:', mean_pca_gluc_mu, '+=', std_pca_gluc_mu)

mean_gluc_mu = float(gluc_mu['mu'].mean())
std_gluc_mu = float(gluc_mu['mu'].std())
print('glucose-MOPS mu:', mean_gluc_mu, '+=', std_gluc_mu)

mean_citr_mu = float(citr_mu['mu'].mean())
std_citr_mu = float(citr_mu['mu'].std())
print('citrat-MOPS mu:', mean_citr_mu, '+=', std_citr_mu)

PCA-Gluc mu: 0.1557936152166388 += 0.012774444510135584
glucose-MOPS mu: 0.5829104732740061 += 0.022196212314551926
citrat-MOPS mu: 0.7229719008169772 += 0.04890741496672533


In [12]:
means = {
    "PCA-Gluc": mean_pca_gluc_mu,
    "glucose-MOPS": mean_gluc_mu,
    "citrat-MOPS": mean_citr_mu
}

stds = {    
    "PCA-Gluc": std_pca_gluc_mu,
    "glucose-MOPS": std_gluc_mu,
    "citrat-MOPS": std_citr_mu
}

models = {
    "PCA-Gluc": "models/iEZ481_PCA_Gluc.xml",
    "glucose-MOPS": "models/iEZ481_Glucose-MOPS.xml",
    "citrat-MOPS": "models/iEZ481_Citrat-MOPS.xml",
}

objectives = {
    "PCA-Gluc": "EX_pca",
    "glucose-MOPS": "EX_glc_e",
    "citrat-MOPS": "EX_cit_e",
}

biomass = "biomass_a"

# from cQuellen2024.ipynb
minimal_uptakes = {
    "PCA-Gluc": -1.714818830406656,
    "glucose-MOPS": -3.796562996954184,
    "citrat-MOPS": -7.447701141488194,
}

n_stds = ["0"]

results = {}

In [33]:
flux_names = []
for case in models.keys():
    print('evaluating', case)
    model = cobra.io.read_sbml_model(models[case])
    flux_names = [rxn.id for rxn in model.reactions]
    model.solver = 'gurobi'
    # max minimizes uptake, because signs are flipped for uptakes.
    model.objective.direction = "max"
    model.objective = 'biomass_a'

    fluxes = []

    for std in n_stds:
        # trick to circumvent warning
        model.reactions.get_by_id(objectives[case]).upper_bound = 0
        model.reactions.get_by_id(objectives[case]).lower_bound = minimal_uptakes[case] 
        # model.reactions.get_by_id('biomass_a').lower_bound = means[case] + int(std)*stds[case]
        # model.reactions.get_by_id('biomass_a').upper_bound = means[case] + int(std)*stds[case]
        solution = model.optimize()
        # print('for mean +', std,'std we obtain:')
        # print(model.summary())
        assert solution.status == 'optimal'
        fluxes.append(solution.fluxes)
        fluxes[-1].rename("mean" + str(std) + "stds", inplace=True)
        print(model.summary())


    fluxes = pd.concat(fluxes, axis=1)
    fluxes.to_csv(f'data/fba_results2/{case}_fluxes.csv')
    print(case)
    print(fluxes)

evaluating PCA-Gluc
Objective
1.0 biomass_a = 0.15579361521663895

Uptake
------
Metabolite Reaction   Flux  C-Number C-Flux
     nh3_e EX_nh3_e  1.197         0  0.00%
      o2_e  EX_o2_e  5.463         0  0.00%
     pca_e   EX_pca  1.715         0  0.00%
      pi_e  EX_pi_e 0.1277         0  0.00%
     so3_e EX_so3_e 0.0187         0  0.00%
       h_c  leakage    8.9         0  0.00%

Secretion
---------
Metabolite Reaction   Flux  C-Number C-Flux
     co2_e EX_co2_e -6.606         0  0.00%
     h2o_e EX_h2o_e -10.67         0  0.00%

PCA-Gluc
         mean0stds
glk       0.000000
pgi      -0.198236
pfkA      0.000000
fbp       0.194518
fda      -0.194518
...            ...
EX_C160   0.000000
EX_C180   0.000000
EX_C161   0.000000
EX_C181   0.000000
quiC      0.000000

[558 rows x 1 columns]
evaluating glucose-MOPS
Objective
1.0 biomass_a = 0.582910473274006

Uptake
------
Metabolite Reaction    Flux  C-Number C-Flux
   glc_D_e EX_glc_e   3.797         0  0.00%
     nh3_e EX_nh3_e   4

In [44]:
old_fba = pd.read_csv(f'data/fba_results/{case}_fluxes.csv')#.set_index('Unnamed: 0')
new_fba = pd.read_csv(f'data/fba_results2/{case}_fluxes.csv')#.set_index('Unnamed: 0')

In [45]:
print(new_fba)
print(old_fba)

    Unnamed: 0  mean0stds
0          glk   0.000000
1          pgi  -0.919927
2         pfkA   0.000000
3          fbp   0.902677
4          fda  -0.902677
..         ...        ...
553    EX_C160   0.000000
554    EX_C180   0.000000
555    EX_C161   0.000000
556    EX_C181   0.000000
557       quiC   0.000000

[558 rows x 2 columns]
    Unnamed: 0  mean0stds
0          glk   0.000000
1          pgi  -0.919927
2         pfkA   0.000000
3          fbp   0.902677
4          fda  -0.902677
..         ...        ...
553    EX_C160   0.000000
554    EX_C180   0.000000
555    EX_C161   0.000000
556    EX_C181   0.000000
557       quiC   0.000000

[558 rows x 2 columns]


In [49]:
print(old_fba['mean0stds'] - new_fba['mean0stds'])
old_fba = old_fba['mean0stds'] 
new_fba = new_fba['mean0stds']

0      0.000000e+00
1      2.275957e-14
2      0.000000e+00
3     -2.264855e-14
4      2.264855e-14
           ...     
553    0.000000e+00
554    0.000000e+00
555    0.000000e+00
556    0.000000e+00
557    0.000000e+00
Name: mean0stds, Length: 558, dtype: float64


In [50]:
print((old_fba - new_fba).max())
print((old_fba - new_fba).mean())
print((old_fba - new_fba).std())
print((old_fba - new_fba).idxmax())

33.355910085602005
0.7146248451144386
4.8762012696163355
276


In [51]:
fluxes_to_plot = ['tkt_2', 'pgi', 'sdhCAB', 'rpi', 'fda', 'sucD', 'gltA', 'pyc', 'pgk', 'aceA',]

In [57]:
indices = []
name = []
for f in fluxes_to_plot:
    indices.append(flux_names.index(f))
indices = sorted(indices)

for i in indices:
    name.append(flux_names[i])

In [60]:
print(name)

['pgi', 'fda', 'pgk', 'gltA', 'sucD', 'sdhCAB', 'aceA', 'pyc', 'rpi', 'tkt_2']


In [63]:
print(old_fba.loc[indices])
print(new_fba.loc[indices])
print(old_fba.loc[indices] - new_fba.loc[indices])

1     -0.919927
4     -0.902677
8     -1.834099
20     0.000000
25     0.000000
26     6.206409
29    25.610428
33     5.875360
38     0.694867
42     0.000000
Name: mean0stds, dtype: float64
1     -0.919927
4     -0.902677
8     -1.834099
20     2.181034
25     8.657010
26    10.371829
29     0.000000
33     0.000000
38     0.694867
42     0.000000
Name: mean0stds, dtype: float64
1     2.275957e-14
4     2.264855e-14
8     4.551914e-14
20   -2.181034e+00
25   -8.657010e+00
26   -4.165420e+00
29    2.561043e+01
33    5.875360e+00
38   -1.421085e-14
42    0.000000e+00
Name: mean0stds, dtype: float64


In [65]:
name[indices.index(25)]

'sucD'